In [1]:
import numpy as np
from scipy.optimize import root_scalar

%pylab inline
import matplotlib.pyplot as plt
from matplotlib import cm
%config InlineBackend.figure_format = 'retina'
plt.style.use('seaborn-whitegrid')
from scipy.interpolate import UnivariateSpline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/var/folders/tw/zmm3b0k97wbfj0cl53y3w2nm0000gn/T/ipykernel_64277/200498476.py:8: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
#------------------------------------------------------------------------------
# EOS and Material Parameters (from ichikawa 2014)
#------------------------------------------------------------------------------

# Number of atoms per formula unit (for monatomic Fe, n=1)
n = 1

# Avogadro's number (atoms/mol)
N_A = 6.02214076e23

# Reference temperature for the isothermal part (in K)
T0 = 7000.0

# Reference volume V0: given as 17.87 Å³/atom. T0 = 7000K and P0 = 0GPa
# Convert: 1 Å³ = 1e-30 m³, then multiply by Avogadro's number to get m³/mol.
V0 = 17.87e-30 * N_A  # m^3/mol
# For reference, V0 ≈ 1.075e-5 m^3/mol

# Isothermal bulk modulus at T0 (in Pa) and its pressure derivative.
# (Typical values for liquid iron are model dependent; use values from Ichikawa et al. (2014).)
K_T0 = 24.6e9    # Example: 100 GPa in Pa
Kp_T0 = 6.65   # Example value

# Grüneisen parameter parameters
gamma0 = 1.85      # Value at V0
b = 0.35           # Exponent

# Thermal energy fitting parameters.
# a is a multiplicative factor (here a = 1)
a = 1.0
# e0 is given as 0.314 in units of 10^{-4}/K, so convert to proper units:
e0 = 0.314e-4  # J/(mol·K^2)
g = -0.4       # exponent for volume scaling in the thermal energy term

# Universal gas constant in J/(mol·K)
R = 8.314

# Molar mass of iron in kg/mol (55.845 g/mol)
M = 0.055845

# Reference temperature (T0) for the isothermal (cold) curve (in K)
T0 = 7000.0

# range for the eos tables: T between 4000-7000 K and P between 60-420 GPa. 
# The calculation presented here is for T between 1000K and 50000 K and P bewteen 0 and 10 TPa. 
# Use at your own risk. 

In [3]:
#------------------------------------------------------------------------------
# EOS Functions
#------------------------------------------------------------------------------

def P_T0(V):
    """
    Isothermal pressure at the reference temperature T0 using the Vinet (Morse-Rydberg) equation:
    
    P_T0(V) = 3 K_T0 (V/V0)^(-2/3) [1 - (V/V0)^(1/3)] * exp{ (3/2)(Kp_T0 - 1)[1 - (V/V0)^(1/3)] }
    """
    eta = (V / V0)**(1/3)
    prefactor = 3 * K_T0 * (V / V0)**(-2/3)
    exponent = (3/2) * (Kp_T0 - 1) * (1 - eta)
    return prefactor * (1 - eta) * np.exp(exponent)

def gamma_V(V):
    """
    Grüneisen parameter as a function of volume:
      gamma(V) = gamma0 * (V/V0)^b
    """
    return gamma0 * (1.0 + a * ((V / V0)**b - 1.0))

def E_th(V, T):
    """
    Thermal internal energy (J/mol) as a function of volume and temperature:
      E_th(V, T) = 3 n R (T + e0 (V/V0)^g T^2).
    """
    return 3.0 * n * R * (T + e0 * (V / V0)**g * T**2.0)

def delta_E_th(V, T):
    """
    Thermal energy difference relative to the reference temperature T0.
    """
    return E_th(V, T) - E_th(V, T0)

def P_total(V, T):
    """
    Total pressure from the EOS:
      P(V, T) = P_T0(V) + [gamma(V)/V] * delta_E_th(V, T)
    """
    return P_T0(V) + (gamma_V(V) / V) * delta_E_th(V, T)

In [4]:
#------------------------------------------------------------------------------
# Finite Difference Derivatives
#------------------------------------------------------------------------------

def dP_dV(V, T, delta=1e-10):
    """
    Compute the partial derivative ∂P/∂V|_T using central finite differences.
    """
    return (P_total(V + delta, T) - P_total(V - delta, T)) / (2 * delta)

def dP_dT(V, T, delta=1e-6):
    """
    Compute the partial derivative ∂P/∂T|_V using central finite differences.
    """
    return (P_total(V, T + delta) - P_total(V, T - delta)) / (2 * delta)


In [5]:
#------------------------------------------------------------------------------
# Specific Heats (Per Mole) and Conversion to Per Mass
#------------------------------------------------------------------------------

def specific_heat_Cv(V, T):
    """
    Specific heat at constant volume on a molar basis:
      C_V = dE_th/dT = 3 n R + 6 n R e0 (V/V0)^g T    (J/(mol·K))
    """
    return 3 * n * R + 6.0 * n * R * e0 * (V / V0)**g * T

def specific_heat_Cp(V, T):
    """
    Specific heat at constant pressure on a molar basis.
    
    Using the thermodynamic relation:
      C_P = C_V + T * ( (∂P/∂T|_V)^2 / (-∂P/∂V|_T) )
    """
    Cv = specific_heat_Cv(V, T)
    dP_dT_val = dP_dT(V, T)
    dP_dV_val = dP_dV(V, T)
    if np.isclose(dP_dV_val, 0.0):
        raise ValueError("dP/dV is too close to zero, cannot compute Cp reliably.")
    return Cv + T * (dP_dT_val**2) / (-dP_dV_val)

def convert_to_per_mass(value_per_mol):
    """
    Convert a quantity from a per-mole basis to a per-mass basis.
    For example, to convert specific heat from J/(mol·K) to J/(kg·K), divide by M.
    """
    return value_per_mol / M

In [6]:
#------------------------------------------------------------------------------
# Additional Thermodynamic Properties: Bulk Modulus and Thermal Expansion
#------------------------------------------------------------------------------

def isothermal_bulk_modulus(V, T):
    """
    Compute the isothermal bulk modulus at constant temperature:
      K_T = -V * (∂P/∂V)_T   (Pa)
    """
    return -V * dP_dV(V, T)

def thermal_expansion_coefficient(V, T):
    """
    Compute the thermal expansion coefficient:
      α = (1/V)(∂V/∂T)_P = (∂P/∂T)_V / K_T   (1/K)
    """
    dP_dT_val = dP_dT(V, T)
    K_T = isothermal_bulk_modulus(V, T)
    if np.isclose(K_T, 0.0):
        raise ValueError("Bulk modulus is zero, cannot compute thermal expansion coefficient.")
    return dP_dT_val / K_T


In [7]:
#------------------------------------------------------------------------------
# Root Finding to Invert the EOS (find V for given P and T)
#------------------------------------------------------------------------------

def residual(V, T, P_target):
    """
    Residual function for root-finding:
      f(V) = P_total(V, T) - P_target = 0.
    """
    return P_total(V, T) - P_target

def find_volume(P_target, T, V_guess):
    """
    Solve for V such that P_total(V, T) = P_target using a bracketing method.
    """
    V_lower = V_guess * 0.5
    V_upper = V_guess * 1.5
    sol = root_scalar(lambda V: residual(V, T, P_target),
                      bracket=[V_lower, V_upper],
                      method='brentq')
    if sol.converged:
        return sol.root
    else:
        raise RuntimeError("Volume root finding did not converge.")



In [8]:
#------------------------------------------------------------------------------
# Compute All Thermodynamic Properties
#------------------------------------------------------------------------------

def compute_properties(P_target, T, rho_guess, T_s0, V_s0):
    """
    Given a target pressure (in Pa) and temperature (in K), solve for the molar
    volume and compute:
      - Density (kg/m^3),
      - Thermal internal energy (J/mol and J/kg),
      - Thermal entropy (J/(mol·K) and J/(kg·K)),
      - Specific heats (C_V and C_P) in J/(mol·K) and J/(kg·K),
      - Isothermal bulk modulus (Pa),
      - Thermal expansion coefficient (1/K).
    """
    # Initial guess for volume (Molar mass divided by initial guess for density to find the guess in molar volume)
    V_guess = M / rho_guess
    V_sol = find_volume(P_target, T, V_guess)
    
    # Density: molar mass divided by molar volume (kg/m^3)
    density = M / V_sol
    
    # Thermal internal energy.
    internal_energy_mol = E_th(V_sol, T)
    internal_energy_mass = convert_to_per_mass(internal_energy_mol)
    
    # Thermal entropy (using S = 3 n R ln(T/T0) + 6 n R e0 (V/V0)^g (T - T0)).
    # T_s0 and V_s0 are the reference state for s = 0
    entropy_mol = 3 * n * R * np.log(T / T_s0) + 6 * n * R * e0 * (V_sol / V_s0)**g * (T - T_s0)
    entropy_mass = convert_to_per_mass(entropy_mol)
    
    # Specific heats.
    Cv_mol = specific_heat_Cv(V_sol, T)
    Cp_mol = specific_heat_Cp(V_sol, T)
    Cv_mass = convert_to_per_mass(Cv_mol)
    Cp_mass = convert_to_per_mass(Cp_mol)
    
    # Isothermal bulk modulus.
    bulk_modulus = isothermal_bulk_modulus(V_sol, T)
    
    # Thermal expansion coefficient.
    alpha = thermal_expansion_coefficient(V_sol, T)
    
    return {
        "molar_volume": V_sol,             # m^3/mol
        "density": density,                # kg/m^3
        "internal_energy_mol": internal_energy_mol,  # J/mol
        "internal_energy_mass": internal_energy_mass,  # J/kg
        "entropy_mol": entropy_mol,        # J/(mol·K)
        "entropy_mass": entropy_mass,      # J/(kg·K)
        "Cv_mol": Cv_mol,                  # J/(mol·K)
        "Cp_mol": Cp_mol,                  # J/(mol·K)
        "Cv_mass": Cv_mass,                # J/(kg·K)
        "Cp_mass": Cp_mass,                # J/(kg·K)
        "bulk_modulus": bulk_modulus,      # Pa
        "thermal_expansion": alpha         # 1/K
    }



In [9]:
def compute_density(P_target, T, rho_guess, T_s0, V_s0):
    """
    Computes density at given pressure (Pa) and temperature (K)
    using the EOS inversion routine.
    """
    try:
        props = compute_properties(P_target, T, rho_guess, T_s0, V_s0)
        return props["density"], props
    except Exception as e:
        # If the root-finding fails, return None to mark that we have no data.
        return None, None


#------------------------------------------------------------------------------
# Main Routine: User Input and Output
#------------------------------------------------------------------------------

if __name__ == "__main__":
    try:
        P_input_GPa = float(input("Enter pressure (GPa): "))
        T_input = float(input("Enter temperature (K): "))
    except ValueError:
        print("Invalid input. Please enter numerical values.")
        exit(1)
    
    # Convert pressure from GPa to Pa (1 GPa = 1e9 Pa)
    P_input = P_input_GPa * 1e9
    
    try:
        props = compute_properties(P_input, T_input)
        print("\n--- Computed Thermodynamic Properties ---")
        print(f"Molar Volume:              {props['molar_volume']:.3e} m^3/mol")
        print(f"Density:                   {props['density']:.3e} kg/m^3")
        print(f"Internal Energy:           {props['internal_energy_mol']:.3e} J/mol "
              f"({props['internal_energy_mass']:.3e} J/kg)")
        print(f"Entropy:                   {props['entropy_mol']:.3e} J/(mol·K) "
              f"({props['entropy_mass']:.3e} J/(kg·K))")
        print(f"Specific Heat (C_V):       {props['Cv_mol']:.3e} J/(mol·K) "
              f"({props['Cv_mass']:.3e} J/(kg·K))")
        print(f"Specific Heat (C_P):       {props['Cp_mol']:.3e} J/(mol·K) "
              f"({props['Cp_mass']:.3e} J/(kg·K))")
        print(f"Isothermal Bulk Modulus:   {props['bulk_modulus']:.3e} Pa")
        print(f"Thermal Expansion Coeff.:  {props['thermal_expansion']:.3e} 1/K")
    except Exception as e:
        print("Error computing properties:", e)

In [84]:
# initialize a pressure grid and 
# a rho_guess grid as the starting guess point for each pressure level with T = T_values[0], T_values defined below

P_grid_GPa = np.linspace(0.0, 10000.0, 1001)
#P_grid_GPa = np.linspace(0.0, 20.0, 201)
rho_guess_grid = np.zeros(len(P_grid_GPa))
# successful_rho saves the amount of successful calculations at each pressure level.
successful_rho = np.zeros(len(P_grid_GPa))

# define the T_grid
# T_values are temperatures where the EoS is known to work
# T_extrap will be used for extrapolation
# initialize to be the same so extrapolation is skipped if not needed

T_values = np.linspace(1000, 50000, 981)
T_extrap = np.linspace(1000, 50000, 981)

# This is the guess for P = P_grid_GPa[0] and T = T_values[0]
initial_rho_guess=6000.0

# for relative entropy, find the V of the reference state, T = 7000.0 and P = 0

T_s0 = T0
#V_guess_s0 = M / initial_rho_guess
#V_s0 = find_volume(P_grid_GPa[0]*1e9, T_s0, V_guess_s0)
V_s0 = V0
#print(f"Temperature and molar volume at the reference state where s = 0: T = ", T_s0, "V = ", V_s0)

# Find initial guess of rho for all pressure levels at T = T0; We picked this as the starting point 
# because there's a solution to rho at every pressure levels in the grid.

T_start = T_values[0]

for i in range(len(P_grid_GPa)): 
    if i == 0:
        rho_guess = initial_rho_guess
    else:
        if rho_guess_grid[i-1] is None:
            rho_guess = initial_rho_guess
        else:
            rho_guess = rho_guess_grid[i-1]
    
    dens, props_list = compute_density(P_grid_GPa[i]*1e9, T_start, rho_guess, T_s0, V_s0)
    if dens is not None:
        rho_guess_grid[i] = dens
    else:
        # If the EOS fails, you might choose to skip or break out of the loop.
        rho_guess_grid[i] = np.nan
        
# perform an spline extrapolation if there is nan in rho_guess_grid ------ need to finish!!!!!!!
mask = np.isfinite(rho_guess_grid)
P_fit = P_grid_GPa[mask]
rho_guess_fit = rho_guess_grid[mask]

if len(rho_guess_fit)==len(rho_guess_grid):
    print('extrapolation for initial guess of rho at each pressure levels not needed.')
#print(len(rho_guess_fit)-len(rho_guess_grid))

extrapolation for initial guess of rho at each pressure levels not needed.


In [85]:
# compute the table

# initialize 2D arrays

rho_table = np.zeros((len(P_grid_GPa), len(T_extrap)))
Eth_table = np.zeros((len(P_grid_GPa), len(T_extrap)))
alpha_table = np.zeros((len(P_grid_GPa), len(T_extrap)))
cP_table = np.zeros((len(P_grid_GPa), len(T_extrap)))
cV_table = np.zeros((len(P_grid_GPa), len(T_extrap)))
s_table = np.zeros((len(P_grid_GPa), len(T_extrap)))
KT_table = np.zeros((len(P_grid_GPa), len(T_extrap)))

In [86]:
for i in range(len(P_grid_GPa)):
    # Choose a fixed pressure level
    P_target_GPa = P_grid_GPa[i]
    P_target = P_target_GPa * 1e9  # convert to Pa

    density_values = []
    entropy_values = []
    alpha_values = []
    Eth_values = []
    cP_values = []
    cV_values = []
    KT_values = []

    # Compute density and other quantities at temperatures where eos probably works.
    for T in T_values:
        if T == T_values[0]:
            rho_guess = rho_guess_grid[i]
        else:
            if np.isfinite(density_values[-1]) == False:
                rho_guess = rho_guess_grid[i]
            else:
                rho_guess = density_values[-1]
        dens, props_list = compute_density(P_target, T, rho_guess, T_s0, V_s0)
        if dens is not None:
            density_values.append(dens)
            entropy_values.append(props_list['entropy_mass'])
            alpha_values.append(props_list["thermal_expansion"])
            Eth_values.append(props_list["internal_energy_mass"])
            cP_values.append(props_list['Cp_mass'])
            cV_values.append(props_list['Cv_mass'])
            KT_values.append(props_list["bulk_modulus"])
        else:
            # If the EOS fails
            density_values.append(np.nan)
            entropy_values.append(np.nan)
            alpha_values.append(np.nan)
            Eth_values.append(np.nan)
            cP_values.append(np.nan)
            cV_values.append(np.nan)
            KT_values.append(np.nan)

    density_values = np.array(density_values)
    entropy_values = np.array(entropy_values)
    alpha_values = np.array(alpha_values)
    Eth_values = np.array(Eth_values)
    cV_values = np.array(cV_values)
    cP_values = np.array(cP_values)
    KT_values = np.array(KT_values)
    # Only use the temperatures where density was computed successfully for extrapolation.
    mask = np.isfinite(density_values)
    T_fit = T_values[mask]
    density_fit = density_values[mask]
    entropy_fit = entropy_values[mask]
    alpha_fit = alpha_values[mask]
    Eth_fit = Eth_values[mask]
    cV_fit = cV_values[mask]
    cP_fit = cP_values[mask]
    KT_fit = KT_values[mask]
    
    # save the amount of successful calculated densities for reference
    successful_rho[i] = len(T_fit)
    
    if len(density_fit) == len(density_values):
        # skip extrapolation if all values are computed successfully
        rho_table[i] = density_values.copy()
        Eth_table[i] = Eth_values.copy()
        alpha_table[i] = alpha_values.copy()
        cP_table[i] = cP_values.copy()
        cV_table[i] = cV_values.copy()
        s_table[i] = entropy_values.copy()
        KT_table[i] = KT_values.copy()
        print('P=',int(P_grid_GPa[i]),'GPa. extrapolation not needed. density:', int(rho_table[i][300]),'kg/m^3, entropy:',int(s_table[i][300]),'J/K/kg, internal energy:', int(Eth_table[i][300]/1e3),'kJ/kg')
    else:
        # perform extrapolation if not all TP levels return real density values
        if len(T_fit) < 2:
            raise ValueError("Not enough data points for extrapolation.")
        # Fit a spline (or use np.polyfit for a polynomial fit).
        # Here we choose a quadratic spline with no smoothing (s=0).
        spline = UnivariateSpline(T_fit, density_fit, k=2, s=0)
        rho_table[i] = spline(T_extrap)

        # For all other quantities
        spline = UnivariateSpline(T_fit, entropy_fit, k=2, s=0)
        s_table[i] = spline(T_extrap)

        spline = UnivariateSpline(T_fit, Eth_fit, k=2, s=0)
        Eth_table[i] = spline(T_extrap)

        spline = UnivariateSpline(T_fit, alpha_fit, k=2, s=0)
        alpha_table[i] = spline(T_extrap)

        spline = UnivariateSpline(T_fit, cV_fit, k=2, s=0)
        cP_table[i] = spline(T_extrap)

        spline = UnivariateSpline(T_fit, cP_fit, k=2, s=0)
        cV_table[i] = spline(T_extrap)

        spline = UnivariateSpline(T_fit, KT_fit, k=2, s=0)
        KT_table[i] = spline(T_extrap)
        
        print('P=',int(P_grid_GPa[i]),'GPa. extrapolation needed! density:', int(rho_table[i][300]),'kg/m^3, entropy:',int(s_table[i][300]),'J/K/kg, internal energy:', int(Eth_table[i][300]/1e3),'kJ/kg')
   

P= 0 GPa. extrapolation needed! density: -509874 kg/m^3, entropy: -579 J/K/kg, internal energy: -29678 kJ/kg
P= 10 GPa. extrapolation not needed. density: 449 kg/m^3, entropy: 464 J/K/kg, internal energy: 8495 kJ/kg
P= 20 GPa. extrapolation not needed. density: 1138 kg/m^3, entropy: 506 J/K/kg, internal energy: 9103 kJ/kg
P= 30 GPa. extrapolation not needed. density: 1964 kg/m^3, entropy: 540 J/K/kg, internal energy: 9580 kJ/kg
P= 40 GPa. extrapolation not needed. density: 2918 kg/m^3, entropy: 569 J/K/kg, internal energy: 9997 kJ/kg
P= 50 GPa. extrapolation not needed. density: 3892 kg/m^3, entropy: 594 J/K/kg, internal energy: 10346 kJ/kg
P= 60 GPa. extrapolation not needed. density: 4733 kg/m^3, entropy: 612 J/K/kg, internal energy: 10606 kJ/kg
P= 70 GPa. extrapolation not needed. density: 5413 kg/m^3, entropy: 625 J/K/kg, internal energy: 10797 kJ/kg
P= 80 GPa. extrapolation not needed. density: 5969 kg/m^3, entropy: 636 J/K/kg, internal energy: 10943 kJ/kg
P= 90 GPa. extrapolation

P= 750 GPa. extrapolation not needed. density: 15218 kg/m^3, entropy: 757 J/K/kg, internal energy: 12667 kJ/kg
P= 760 GPa. extrapolation not needed. density: 15289 kg/m^3, entropy: 758 J/K/kg, internal energy: 12677 kJ/kg
P= 770 GPa. extrapolation not needed. density: 15359 kg/m^3, entropy: 758 J/K/kg, internal energy: 12687 kJ/kg
P= 780 GPa. extrapolation not needed. density: 15429 kg/m^3, entropy: 759 J/K/kg, internal energy: 12697 kJ/kg
P= 790 GPa. extrapolation not needed. density: 15498 kg/m^3, entropy: 760 J/K/kg, internal energy: 12707 kJ/kg
P= 800 GPa. extrapolation not needed. density: 15567 kg/m^3, entropy: 760 J/K/kg, internal energy: 12717 kJ/kg
P= 810 GPa. extrapolation not needed. density: 15634 kg/m^3, entropy: 761 J/K/kg, internal energy: 12727 kJ/kg
P= 820 GPa. extrapolation not needed. density: 15702 kg/m^3, entropy: 762 J/K/kg, internal energy: 12736 kJ/kg
P= 830 GPa. extrapolation not needed. density: 15769 kg/m^3, entropy: 762 J/K/kg, internal energy: 12746 kJ/kg
P

P= 1520 GPa. extrapolation not needed. density: 19536 kg/m^3, entropy: 798 J/K/kg, internal energy: 13247 kJ/kg
P= 1530 GPa. extrapolation not needed. density: 19582 kg/m^3, entropy: 798 J/K/kg, internal energy: 13252 kJ/kg
P= 1540 GPa. extrapolation not needed. density: 19628 kg/m^3, entropy: 799 J/K/kg, internal energy: 13258 kJ/kg
P= 1550 GPa. extrapolation not needed. density: 19674 kg/m^3, entropy: 799 J/K/kg, internal energy: 13264 kJ/kg
P= 1560 GPa. extrapolation not needed. density: 19719 kg/m^3, entropy: 799 J/K/kg, internal energy: 13270 kJ/kg
P= 1570 GPa. extrapolation not needed. density: 19764 kg/m^3, entropy: 800 J/K/kg, internal energy: 13275 kJ/kg
P= 1580 GPa. extrapolation not needed. density: 19810 kg/m^3, entropy: 800 J/K/kg, internal energy: 13281 kJ/kg
P= 1590 GPa. extrapolation not needed. density: 19855 kg/m^3, entropy: 800 J/K/kg, internal energy: 13286 kJ/kg
P= 1600 GPa. extrapolation not needed. density: 19899 kg/m^3, entropy: 801 J/K/kg, internal energy: 1329

P= 2280 GPa. extrapolation not needed. density: 22638 kg/m^3, entropy: 824 J/K/kg, internal energy: 13617 kJ/kg
P= 2290 GPa. extrapolation not needed. density: 22675 kg/m^3, entropy: 824 J/K/kg, internal energy: 13621 kJ/kg
P= 2300 GPa. extrapolation not needed. density: 22711 kg/m^3, entropy: 824 J/K/kg, internal energy: 13626 kJ/kg
P= 2310 GPa. extrapolation not needed. density: 22747 kg/m^3, entropy: 825 J/K/kg, internal energy: 13630 kJ/kg
P= 2320 GPa. extrapolation not needed. density: 22784 kg/m^3, entropy: 825 J/K/kg, internal energy: 13634 kJ/kg
P= 2330 GPa. extrapolation not needed. density: 22820 kg/m^3, entropy: 825 J/K/kg, internal energy: 13638 kJ/kg
P= 2340 GPa. extrapolation not needed. density: 22856 kg/m^3, entropy: 826 J/K/kg, internal energy: 13642 kJ/kg
P= 2350 GPa. extrapolation not needed. density: 22892 kg/m^3, entropy: 826 J/K/kg, internal energy: 13646 kJ/kg
P= 2360 GPa. extrapolation not needed. density: 22928 kg/m^3, entropy: 826 J/K/kg, internal energy: 1365

P= 3060 GPa. extrapolation not needed. density: 25262 kg/m^3, entropy: 844 J/K/kg, internal energy: 13907 kJ/kg
P= 3070 GPa. extrapolation not needed. density: 25293 kg/m^3, entropy: 844 J/K/kg, internal energy: 13911 kJ/kg
P= 3080 GPa. extrapolation not needed. density: 25324 kg/m^3, entropy: 845 J/K/kg, internal energy: 13914 kJ/kg
P= 3090 GPa. extrapolation not needed. density: 25355 kg/m^3, entropy: 845 J/K/kg, internal energy: 13917 kJ/kg
P= 3100 GPa. extrapolation not needed. density: 25386 kg/m^3, entropy: 845 J/K/kg, internal energy: 13921 kJ/kg
P= 3110 GPa. extrapolation not needed. density: 25416 kg/m^3, entropy: 845 J/K/kg, internal energy: 13924 kJ/kg
P= 3120 GPa. extrapolation not needed. density: 25447 kg/m^3, entropy: 846 J/K/kg, internal energy: 13927 kJ/kg
P= 3130 GPa. extrapolation not needed. density: 25478 kg/m^3, entropy: 846 J/K/kg, internal energy: 13930 kJ/kg
P= 3140 GPa. extrapolation not needed. density: 25509 kg/m^3, entropy: 846 J/K/kg, internal energy: 1393

...

P= 6860 GPa. extrapolation not needed. density: 34634 kg/m^3, entropy: 908 J/K/kg, internal energy: 14817 kJ/kg
P= 6870 GPa. extrapolation not needed. density: 34654 kg/m^3, entropy: 908 J/K/kg, internal energy: 14819 kJ/kg
P= 6880 GPa. extrapolation not needed. density: 34675 kg/m^3, entropy: 908 J/K/kg, internal energy: 14821 kJ/kg
P= 6890 GPa. extrapolation not needed. density: 34695 kg/m^3, entropy: 908 J/K/kg, internal energy: 14822 kJ/kg
P= 6900 GPa. extrapolation not needed. density: 34715 kg/m^3, entropy: 909 J/K/kg, internal energy: 14824 kJ/kg
P= 6910 GPa. extrapolation not needed. density: 34736 kg/m^3, entropy: 909 J/K/kg, internal energy: 14826 kJ/kg
P= 6920 GPa. extrapolation not needed. density: 34756 kg/m^3, entropy: 909 J/K/kg, internal energy: 14828 kJ/kg
P= 6930 GPa. extrapolation not needed. density: 34777 kg/m^3, entropy: 909 J/K/kg, internal energy: 14830 kJ/kg
P= 6940 GPa. extrapolation not needed. density: 34797 kg/m^3, entropy: 909 J/K/kg, internal energy: 1483

P= 7610 GPa. extrapolation not needed. density: 36131 kg/m^3, entropy: 917 J/K/kg, internal energy: 14948 kJ/kg
P= 7620 GPa. extrapolation not needed. density: 36151 kg/m^3, entropy: 917 J/K/kg, internal energy: 14950 kJ/kg
P= 7630 GPa. extrapolation not needed. density: 36170 kg/m^3, entropy: 918 J/K/kg, internal energy: 14951 kJ/kg
P= 7640 GPa. extrapolation not needed. density: 36190 kg/m^3, entropy: 918 J/K/kg, internal energy: 14953 kJ/kg
P= 7650 GPa. extrapolation not needed. density: 36209 kg/m^3, entropy: 918 J/K/kg, internal energy: 14955 kJ/kg
P= 7660 GPa. extrapolation not needed. density: 36228 kg/m^3, entropy: 918 J/K/kg, internal energy: 14956 kJ/kg
P= 7670 GPa. extrapolation not needed. density: 36248 kg/m^3, entropy: 918 J/K/kg, internal energy: 14958 kJ/kg
P= 7680 GPa. extrapolation not needed. density: 36267 kg/m^3, entropy: 918 J/K/kg, internal energy: 14960 kJ/kg
P= 7690 GPa. extrapolation not needed. density: 36287 kg/m^3, entropy: 918 J/K/kg, internal energy: 1496

P= 8360 GPa. extrapolation not needed. density: 37558 kg/m^3, entropy: 926 J/K/kg, internal energy: 15070 kJ/kg
P= 8370 GPa. extrapolation not needed. density: 37577 kg/m^3, entropy: 926 J/K/kg, internal energy: 15071 kJ/kg
P= 8380 GPa. extrapolation not needed. density: 37596 kg/m^3, entropy: 926 J/K/kg, internal energy: 15073 kJ/kg
P= 8390 GPa. extrapolation not needed. density: 37614 kg/m^3, entropy: 926 J/K/kg, internal energy: 15075 kJ/kg
P= 8400 GPa. extrapolation not needed. density: 37633 kg/m^3, entropy: 926 J/K/kg, internal energy: 15076 kJ/kg
P= 8410 GPa. extrapolation not needed. density: 37651 kg/m^3, entropy: 926 J/K/kg, internal energy: 15078 kJ/kg
P= 8420 GPa. extrapolation not needed. density: 37670 kg/m^3, entropy: 927 J/K/kg, internal energy: 15079 kJ/kg
P= 8430 GPa. extrapolation not needed. density: 37688 kg/m^3, entropy: 927 J/K/kg, internal energy: 15081 kJ/kg
P= 8440 GPa. extrapolation not needed. density: 37707 kg/m^3, entropy: 927 J/K/kg, internal energy: 1508

P= 9140 GPa. extrapolation not needed. density: 38978 kg/m^3, entropy: 934 J/K/kg, internal energy: 15188 kJ/kg
P= 9150 GPa. extrapolation not needed. density: 38996 kg/m^3, entropy: 934 J/K/kg, internal energy: 15190 kJ/kg
P= 9160 GPa. extrapolation not needed. density: 39014 kg/m^3, entropy: 934 J/K/kg, internal energy: 15191 kJ/kg
P= 9170 GPa. extrapolation not needed. density: 39032 kg/m^3, entropy: 935 J/K/kg, internal energy: 15193 kJ/kg
P= 9180 GPa. extrapolation not needed. density: 39049 kg/m^3, entropy: 935 J/K/kg, internal energy: 15194 kJ/kg
P= 9190 GPa. extrapolation not needed. density: 39067 kg/m^3, entropy: 935 J/K/kg, internal energy: 15196 kJ/kg
P= 9200 GPa. extrapolation not needed. density: 39085 kg/m^3, entropy: 935 J/K/kg, internal energy: 15197 kJ/kg
P= 9210 GPa. extrapolation not needed. density: 39103 kg/m^3, entropy: 935 J/K/kg, internal energy: 15199 kJ/kg
P= 9220 GPa. extrapolation not needed. density: 39120 kg/m^3, entropy: 935 J/K/kg, internal energy: 1520

P= 9890 GPa. extrapolation not needed. density: 40290 kg/m^3, entropy: 942 J/K/kg, internal energy: 15295 kJ/kg
P= 9900 GPa. extrapolation not needed. density: 40307 kg/m^3, entropy: 942 J/K/kg, internal energy: 15297 kJ/kg
P= 9910 GPa. extrapolation not needed. density: 40324 kg/m^3, entropy: 942 J/K/kg, internal energy: 15298 kJ/kg
P= 9920 GPa. extrapolation not needed. density: 40341 kg/m^3, entropy: 942 J/K/kg, internal energy: 15300 kJ/kg
P= 9930 GPa. extrapolation not needed. density: 40358 kg/m^3, entropy: 942 J/K/kg, internal energy: 15301 kJ/kg
P= 9940 GPa. extrapolation not needed. density: 40375 kg/m^3, entropy: 942 J/K/kg, internal energy: 15302 kJ/kg
P= 9950 GPa. extrapolation not needed. density: 40392 kg/m^3, entropy: 942 J/K/kg, internal energy: 15304 kJ/kg
P= 9960 GPa. extrapolation not needed. density: 40409 kg/m^3, entropy: 942 J/K/kg, internal energy: 15305 kJ/kg
P= 9970 GPa. extrapolation not needed. density: 40427 kg/m^3, entropy: 943 J/K/kg, internal energy: 1530

In [51]:
# save files 

np.savetxt('rho_Fel_ichikawa2014.txt',rho_table)
np.savetxt('s_Fel_ichikawa2014.txt',s_table)
np.savetxt('Eth_Fel_ichikawa2014.txt',Eth_table)
np.savetxt('alpha_Fel_ichikawa2014.txt',alpha_table)
np.savetxt('cV_Fel_ichikawa2014.txt',cV_table)
np.savetxt('cP_Fel_ichikawa2014.txt',cP_table)
np.savetxt('KT_Fel_ichikawa2014.txt',KT_table)
np.savetxt('Pgrid_Fel_ichikawa2014.txt',np.transpose([P_grid_GPa*1e9]))
np.savetxt('Tgrid_Fel_ichikawa2014.txt',np.transpose([T_extrap]))

In [ ]:
######### not quite sure. 

###### need to think about how to find S0
# S0 doesn't technically exsit for liquid Fe. 

def S_function(V, T, T_ref, S0):
    """
    Returns the entropy S(V,T) with an arbitrary integration constant S0.
    
    S(V,T) = 3 n R ln(T/T_ref) + 2 a e0 (V/V0)^g (T - T_ref) + S0.
    """
    return 3 * n * R * math.log(T / T_ref) + 2 * a * e0 * (V / V0)**g * (T - T_ref) + S0

# Suppose we choose T_ref = 7000 K (as before) and we wish to calibrate S such that
# S(V*, T*) = 0 at some reference state (V*, T*). For example, if you have a reference state
# from experiment or another theoretical calculation, set:
T_star = 5000.0       # for example, 5000 K
V_star = 1.0e-5       # for example, 1.0e-5 m^3/mol
# Determine S0 so that S(V_star, T_star)=0:
S0 = - (3 * n * R * math.log(T_star / 7000.0) +
        2 * a * e0 * (V_star / V0)**g * (T_star - 7000.0))

print("Integration constant S0 set to:", S0, "J/(mol·K)")

# Now, if you want to find for a given V the temperature T_zero where S(V,T_zero)=0,
# define a function of T for a fixed V.
def find_T_zero(V, T_ref, S0, T_low=1.0, T_high=1e5):
    """
    Finds T such that S(V, T) = 0 for given V, using S_function.
    """
    f = lambda T: S_function(V, T, T_ref, S0)
    sol = root_scalar(f, bracket=[T_low, T_high], method='brentq')
    if sol.converged:
        return sol.root
    else:
        raise RuntimeError("Failed to find T_zero for S=0")

# For example, for V = V_star, find T_zero:
T_zero = find_T_zero(V_star, 7000.0, S0)
print("For V =", V_star, "m^3/mol, the temperature where S=0 is approximately", T_zero, "K")